This file outlines a comprehensive approach for building a conversational agent that leverages Natural Language Processing (NLP) techniques and neural network models, specifically BERT (Bidirectional Encoder Representations from Transformers).

THIS IS AN EXPERIMENTAL FILE.



I HAVE MADE THIS  FILE FROM  MY UNDERSTANDING AND LEARNING ABOUT THE LLM's 

In [4]:
pip install nltk transformers torch


Setting Up NLTK Components
Preprocessing Text
Reading and Processing Lecture Notes

In [5]:
import os
import nltk
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import torch
from transformers import BertTokenizer, BertModel

#  NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):

    text = text.lower()
    

    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    

    sentences = sent_tokenize(text)

    processed_sentences = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        words = [stemmer.stem(word) for word in words]
        words = [lemmatizer.lemmatize(word) for word in words]
        processed_sentences.append(' '.join(words))
    
    return processed_sentences

# Directory .txt files
directory = 'lecture_notes'


lecture_texts = {}
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            lecture_name = filename.replace('.txt', '')
            text = file.read()
            lecture_texts[lecture_name] = preprocess_text(text)

# Print 
for lecture_name, processed_text in lecture_texts.items():
    print(f"Lecture: {lecture_name}")
    for sentence in processed_text[:5]:
        print(sentence)
    print('-' * 40)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mamid\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mamid\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Lecture: capabilities
lectur capabl in thi lectur we will explor the capabl of gpt3 the canon larg languag model we will close follow the benchmark from the gpt3 paper which includ standard nlp benchmark eg question answer a well a quitki oneoff demo eg use a new word in a sentenc in comparison with the stateoftheartresult for each task the result are mix on some task such a languag model gpt3 exce the stateoftheart by a huge margin onoth where gpt3 is compet against system that are train with larg amount of label data it lag far behind the way to think about these result is a follow gpt3 wa not train on these task explicitli it wa just train a a languag model to predict the next word nonetheless even without tri gpt3 doe a passabl job on averag at a broad rang of nlp task becaus gpt3 wa not train on ani of these task it hasnt overfit which mean it ha a good chanc of do well at mani mani other task a seen by the passabl perform on oneoff task moreov if you want to do well on ani partic

 Vectorizing Text with BERT

In [6]:
# pre-trained BERT model 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def vectorize_text(sentences):
    vectors = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        vectors.append(outputs.last_hidden_state.mean(dim=1).squeeze().numpy())
    return vectors

# Vectorize 
lecture_vectors = {}
for lecture_name, processed_text in lecture_texts.items():
    lecture_vectors[lecture_name] = vectorize_text(processed_text)

# Print 
for lecture_name, vectors in lecture_vectors.items():
    print(f"Lecture: {lecture_name}, Vector Shape: {vectors[0].shape}")


Lecture: capabilities, Vector Shape: (768,)
Lecture: Harms I, Vector Shape: (768,)
Lecture: Harms II, Vector Shape: (768,)
Lecture: intro, Vector Shape: (768,)
Lecture: table, Vector Shape: (768,)


In [7]:
pip install faiss-cpu


Note: you may need to restart the kernel to use updated packages.


Indexing Vectors with FAISS

FAISS (Facebook AI Similarity Search) is used to index these vectors for efficient similarity searches. The vectors are added to a FAISS index, which is then saved to a file (lecture_notes.index).

In [8]:
import faiss
import numpy as np

# Create a FAISS index
dimension = 768  # Dimension 
index = faiss.IndexFlatL2(dimension)  

all_vectors = []
text_chunks = []

for lecture_name, vectors in lecture_vectors.items():
    all_vectors.extend(vectors)
    text_chunks.extend([(lecture_name, i, sentence) for i, sentence in enumerate(lecture_texts[lecture_name])])


all_vectors = np.array(all_vectors).astype('float32')


index.add(all_vectors)


print(f"Total vectors indexed: {index.ntotal}")

faiss.write_index(index, 'lecture_notes.index')


Total vectors indexed: 5


Storing Text and Vectors in SQLite

In [9]:
import sqlite3

# SQLite database
conn = sqlite3.connect('lecture_notes.db')
c = conn.cursor()


c.execute('''
    CREATE TABLE IF NOT EXISTS TextChunks (
        id INTEGER PRIMARY KEY,
        lecture_name TEXT,
        chunk_index INTEGER,
        text_chunk TEXT
    )
''')


c.execute('''
    CREATE TABLE IF NOT EXISTS Vectors (
        id INTEGER PRIMARY KEY,
        chunk_id INTEGER,
        vector BLOB,
        FOREIGN KEY (chunk_id) REFERENCES TextChunks (id)
    )
''')


for lecture_name, processed_text in lecture_texts.items():
    for i, text_chunk in enumerate(processed_text):
        c.execute('INSERT INTO TextChunks (lecture_name, chunk_index, text_chunk) VALUES (?, ?, ?)', (lecture_name, i, text_chunk))


conn.commit()

c.execute('SELECT id FROM TextChunks')
chunk_ids = c.fetchall()

for chunk_id, vector in zip(chunk_ids, all_vectors):
    vector_blob = vector.tobytes()
    c.execute('INSERT INTO Vectors (chunk_id, vector) VALUES (?, ?)', (chunk_id[0], vector_blob))


conn.commit()
conn.close()


Querying and Searching

In [10]:
import sqlite3
import numpy as np


conn = sqlite3.connect('lecture_notes.db')
c = conn.cursor()

# Fetch and print 
c.execute('SELECT * FROM TextChunks LIMIT 5')
text_chunks = c.fetchall()
print("Sample text chunks:")
for chunk in text_chunks:
    print(chunk)

# Fetch and print  vectors
c.execute('SELECT * FROM Vectors LIMIT 5')
vectors = c.fetchall()
print("\nSample vectors:")
for vector in vectors:
    chunk_id, vector_blob = vector[1], vector[2]
    vector_array = np.frombuffer(vector_blob, dtype='float32')
    print(f"Chunk ID: {chunk_id}, Vector Shape: {vector_array.shape}")

conn.close()


Sample text chunks:
(1, 'capabilities', 0, 'lectur capabl in thi lectur we will explor the capabl of gpt3 the canon larg languag model we will close follow the benchmark from the gpt3 paper which includ standard nlp benchmark eg question answer a well a quitki oneoff demo eg use a new word in a sentenc in comparison with the stateoftheartresult for each task the result are mix on some task such a languag model gpt3 exce the stateoftheart by a huge margin onoth where gpt3 is compet against system that are train with larg amount of label data it lag far behind the way to think about these result is a follow gpt3 wa not train on these task explicitli it wa just train a a languag model to predict the next word nonetheless even without tri gpt3 doe a passabl job on averag at a broad rang of nlp task becaus gpt3 wa not train on ani of these task it hasnt overfit which mean it ha a good chanc of do well at mani mani other task a seen by the passabl perform on oneoff task moreov if you want to

In [11]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def vectorize_query(query):
    inputs = tokenizer(query, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    query_vector = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return query_vector


user_query = "What are the benefits of large language models?"


query_vector = vectorize_query(user_query)

# Print 
print("Query Vector Shape:", query_vector.shape)


Query Vector Shape: (768,)


In [12]:
import faiss

# FAISS index
index = faiss.read_index('lecture_notes.index')

def search_similar_chunks(query_vector, k=5):
    distances, indices = index.search(np.array([query_vector]), k)
    return distances[0], indices[0]


distances, indices = search_similar_chunks(query_vector, k=5)

# Print 
print("Distances:", distances)
print("Indices:", indices)


Distances: [75.52626  75.627716 79.10797  84.85488  88.42491 ]
Indices: [3 2 0 1 4]


In [13]:
# Adjusted function to retrieve text chunks
def retrieve_text_chunks(indices):
    retrieved_chunks = []
    for idx in indices:
        if len(text_chunks[idx]) >= 3:  # Ensure the tuple has at least 3 values
            lecture_name, chunk_index, text_chunk = text_chunks[idx][:3]  # Slice the tuple to extract the first three values
            retrieved_chunks.append((lecture_name, chunk_index, str(text_chunk)))  # Convert chunk_index to string
        else:
            print(f"Skipping index {idx} as the tuple does not have enough values.")
    return retrieved_chunks


In [14]:

retrieved_chunks = retrieve_text_chunks(indices)

# Print the retrieved text chunks
print("Retrieved Text Chunks:")
for lecture_name, chunk_index, text_chunk in retrieved_chunks:
    print(f"Lecture: {lecture_name}\nChunk Index: {chunk_index}\nText Chunk: {text_chunk}\n")

# Adjusted function to generate answer
def generate_answer(retrieved_chunks):
    answer = ""
    for _, _, text_chunk in retrieved_chunks:
        answer += text_chunk + " "
    return answer.strip()


generated_answer = generate_answer(retrieved_chunks)

print("Generated Answer:")
print(generated_answer)


Retrieved Text Chunks:
Lecture: 4
Chunk Index: intro
Text Chunk: 0

Lecture: 3
Chunk Index: Harms II
Text Chunk: 0

Lecture: 1
Chunk Index: capabilities
Text Chunk: 0

Lecture: 2
Chunk Index: Harms I
Text Chunk: 0

Lecture: 5
Chunk Index: table
Text Chunk: 0

Generated Answer:
0 0 0 0 0


Conversational Agent Class

A ConversationalAgent class is implemented to manage the conversation. It can add user queries and responses to its memory, generate follow-up responses based on the conversation context, and clear its memory.

In [15]:
class ConversationalAgent:
    def __init__(self):
        self.memory = []

    def add_to_memory(self, user_query, response):
        self.memory.append({'query': user_query, 'response': response})

    def get_memory(self):
        return self.memory

    def clear_memory(self):
        self.memory = []


agent = ConversationalAgent()
agent.add_to_memory("What are the benefits of large language models?", "Large language models provide...")
agent.add_to_memory("How do they work?", "They work by...")

for entry in agent.get_memory():
    print(f"User Query: {entry['query']}\nResponse: {entry['response']}\n")


User Query: What are the benefits of large language models?
Response: Large language models provide...

User Query: How do they work?
Response: They work by...



In [16]:
def generate_answer_with_citations(retrieved_chunks):
    answer = ""
    for lecture_name, chunk_index, text_chunk in retrieved_chunks:
        citation = f"[{lecture_name}, Chunk {chunk_index}]"
        answer += f"{text_chunk} ({citation}) "
    return answer.strip()


generated_answer_with_citations = generate_answer_with_citations(retrieved_chunks)


print("Generated Answer with Citations:")
print(generated_answer_with_citations)


Generated Answer with Citations:
0 ([4, Chunk intro]) 0 ([3, Chunk Harms II]) 0 ([1, Chunk capabilities]) 0 ([2, Chunk Harms I]) 0 ([5, Chunk table])


In [17]:
class ConversationalAgent:
    def __init__(self):
        self.memory = []

    def add_to_memory(self, user_query, response):
        self.memory.append({'query': user_query, 'response': response})

    def get_memory(self):
        return self.memory

    def clear_memory(self):
        self.memory = []

    def generate_follow_up_response(self, user_query):

        context = " ".join([entry['response'] for entry in self.memory])
        combined_query = context + " " + user_query
        
        query_vector = vectorize_query(combined_query)
        distances, indices = search_similar_chunks(query_vector, k=5)
        retrieved_chunks = retrieve_text_chunks(indices)
        response = generate_answer_with_citations(retrieved_chunks)

        self.add_to_memory(user_query, response)
        return response


agent = ConversationalAgent()
response1 = agent.generate_follow_up_response("What are the benefits of large language models?")
print(response1)
response2 = agent.generate_follow_up_response("How do they work?")
print(response2)


0 ([4, Chunk intro]) 0 ([3, Chunk Harms II]) 0 ([1, Chunk capabilities]) 0 ([2, Chunk Harms I]) 0 ([5, Chunk table])
0 ([4, Chunk intro]) 0 ([1, Chunk capabilities]) 0 ([3, Chunk Harms II]) 0 ([5, Chunk table]) 0 ([2, Chunk Harms I])


In [18]:
class MultiClassConversationalAgent:
    def __init__(self):
        self.indices = {}
        self.text_chunks = {}

    def add_class(self, class_name, vectors, text_chunks):
        dimension = vectors.shape[1]
        index = faiss.IndexFlatL2(dimension)
        index.add(vectors)
        self.indices[class_name] = index
        self.text_chunks[class_name] = text_chunks

    def search_class(self, class_name, query_vector, k=5):
        index = self.indices[class_name]
        distances, indices = index.search(np.array([query_vector]), k)
        return distances, indices

    def retrieve_text_chunks(self, class_name, indices):
        text_chunks = self.text_chunks[class_name]
        retrieved_chunks = []
        for idx in indices[0]:
            *values, text_chunk = text_chunks[idx]
            retrieved_chunks.append((*values[:2], text_chunk))
        return retrieved_chunks


multi_agent = MultiClassConversationalAgent()
multi_agent.add_class('CS324', np.array(all_vectors).astype('float32'), text_chunks)
query_vector = vectorize_query("What is a large language model")
distances, indices = multi_agent.search_class('CS324', query_vector, k=5)
retrieved_chunks = multi_agent.retrieve_text_chunks('CS324', indices)


generated_answer = generate_answer_with_citations(retrieved_chunks)
print("Generated Answer for CS324:")
print(generated_answer)


Generated Answer for CS324:
what is a languag model the classic definit of a languag model lm is a probabl distribut over sequenc of token suppos we have a vocabulari v of a set of token a languag model p assign each sequenc of token x1xlv a probabl a number between 0 and 1 px1xl the probabl intuit tell u how good a sequenc of token is for exampl if the vocabulari is vateballcheesemouseth the languag model might assign demo pthemouseatethecheese002 pthecheeseatethemouse001 pmousethethecheeseate00001 mathemat a languag model is a veri simpl and beauti object but the simplic is deceiv the abil to assign meaning probabl to all sequenc requir extraordinari but implicit linguist abil and world knowledg for exampl the lm should assign mous the the chees ate a veri low probabl implicitli becaus it ungrammat syntact knowledg the lm should assign the mous ate the chees higher probabl than the chees ate the mous implicitli becaus of world knowledg both sentenc are the same syntact but they diffe

Testing 

In [21]:
import unittest
import subprocess


def run_tests():
 
    result = subprocess.run(['python', 'test_lecture_processing.py'], capture_output=True, text=True)

    # Print 
    print("Test Output:")
    print(result.stdout)
    print("Test Errors:")
    print(result.stderr)

    if result.returncode == 0:
        print("Tests Passed Successfully")
    else:
        print("Tests Failed")

if __name__ == "__main__":
 
    run_tests()



Test Output:

Test Errors:
2024-06-15 12:40:25.616736: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-15 12:40:26.652345: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mamid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mamid\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mam